In [1]:
import xarray as xr
import os
import glob
import numpy as np
import pandas as pd
import dask.dataframe as dd
from distributed import Client
import dask

/glade/u/home/mortimer/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/glade/u/home/mortimer/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


https://github.com/pydata/xarray/pull/2706

In [3]:
#only to be run when new files needed
#files = glob.glob('/glade/scratch/mortimer/argo/**/*_prof.nc',recursive=True)
#pd.DataFrame(files,columns=['files']).sort_values('files').to_csv('/glade/scratch/mortimer/argo/catalogue.csv',index=None)

In [2]:
dffiles = pd.read_csv('/glade/scratch/mortimer/argo/catalogue.csv')
files = list(dffiles.files)

In [51]:
client = Client(processes=False)
client

Client Scheduler: inproc://128.117.181.207/5589/18 Dashboard: http://localhost:38907/status,Cluster Workers: 1 Cores: 72 Memory: 269.89 GB


In [50]:
client.close()

In [52]:
from functools import partial

data_types ={'CONFIG_MISSION_NUMBER':'float64','CYCLE_NUMBER':'float64','DATA_CENTRE':'|U2','DATA_MODE':'|U1',
             'DATA_STATE_INDICATOR':'|U4','DC_REFERENCE':'|U32','DIRECTION':'|U1','FIRMWARE_VERSION':'|U32',
             'FLOAT_SERIAL_NO':'|U32','JULD':'float64','JULD_LOCATION':'float64','JULD_QC':'|U1','LATITUDE':'float64',
             'LONGITUDE':'float64','PI_NAME':'|U64','PLATFORM_NUMBER':'|U8','PLATFORM_TYPE':'|U32','POSITIONING_SYSTEM':'|U8',
             'POSITION_QC':'|U1','PRES':'float64','PRES_ADJUSTED':'float64','PRES_ADJUSTED_ERROR':'float64',
             'PRES_ADJUSTED_QC':'|U1','PRES_QC':'|U1','PROFILE_PRES_QC':'|U1','PROFILE_PSAL_QC':'|U1','PROFILE_TEMP_QC':'|U1',
             'PROJECT_NAME':'|U64','PSAL':'float64','PSAL_ADJUSTED':'float64','PSAL_ADJUSTED_ERROR':'float64',
             'PSAL_ADJUSTED_QC':'|U1','PSAL_QC':'|U1','TEMP':'float64','TEMP_ADJUSTED':'float64','TEMP_ADJUSTED_ERROR':'float64',
             'TEMP_ADJUSTED_QC':'|U1','TEMP_QC':'|U1','VERTICAL_SAMPLING_SCHEME':'|U256','WMO_INST_TYPE':'|U4'}

data_levels =['PRES','PRES_ADJUSTED','PRES_ADJUSTED_ERROR','PRES_ADJUSTED_QC','PRES_QC','PSAL','PSAL_ADJUSTED',
              'PSAL_ADJUSTED_ERROR','PSAL_ADJUSTED_QC','PSAL_QC','TEMP','TEMP_ADJUSTED','TEMP_ADJUSTED_ERROR',
              'TEMP_ADJUSTED_QC','TEMP_QC']

def process_mf(dsinput,levels,data_types=data_types,data_levels=data_levels):
    ds = xr.Dataset()
    #pad =xr.DataArray(np.ones((levels-len( dsinput.N_LEVELS),len(dsinput.N_PROF))) *np.nan,dims={'N_LEVELS','N_PROF'})
    #pad_qc = xr.DataArray(np.chararray((levels-len( dsinput.N_LEVELS),len(dsinput.N_PROF))),dims={'N_LEVELS','N_PROF'})
    dims =('N_PROF','N_LEVELS')
    pading =xr.DataArray(np.ones((len(dsinput.N_PROF),levels-len( dsinput.N_LEVELS))) *np.nan,dims=dims)
    pad_qc = xr.DataArray(np.chararray((len(dsinput.N_PROF),levels-len( dsinput.N_LEVELS))),dims=dims)
    pad_qc[:] = b' '
    for varname in data_types.keys():
        if varname in dsinput.data_vars:
            da = dsinput[varname]
            if 'N_LEVELS' in da.dims:   
                if varname in dsinput.data_vars:
                    if varname.endswith('QC'):
                        da = xr.concat([dsinput[varname],pad_qc],dim='N_LEVELS').astype(data_types[varname])
                    else:
                        da = xr.concat([dsinput[varname],pading],dim='N_LEVELS').astype(data_types[varname])
            else:
                da = dsinput[varname].astype(data_types[varname])
        else:
            if varname in data_levels:
                if data_types[varname]=='float64':
                    da = xr.DataArray(np.ones((len(dsinput.N_PROF),levels))*np.nan , name=varname, dims=['N_PROF','N_LEVELS'])
                else:
                    p=np.chararray((len(dsinput.N_PROF),levels))
                    p[:]=b'0'
                    da = xr.DataArray(p.astype(data_types[varname]), name=varname, dims=['N_PROF','N_LEVELS'])
            else:
                if data_types[varname]=='float64':
                    da = xr.DataArray(np.ones(len(dsinput.N_PROF))*np.nan , name=varname, dims=['N_PROF'])
                else:
                    p=np.chararray((len(dsinput.N_PROF)))
                    p[:]=b'0'
                    da = xr.DataArray(p.astype(data_types[varname]), name=varname, dims=['N_PROF'])
        if not ('HISTORY' in varname) and ('N_CALIB' not in da.dims) and ('N_PARAM' not in da.dims) and  ('N_PROF' in da.dims):
                ds[varname]= da
    return ds
   
preproc = partial(process_mf,levels=3000)

@dask.delayed
def process_float(file):
    data = preproc(xr.open_dataset(file))
    data =data.chunk({'N_PROF':100,'N_LEVELS':3000})
    return data


# Set up the zar file

In [53]:

import zarr
import os
from tqdm import tqdm_notebook
import numcodecs
numcodecs.blosc.use_threads = False
#synchronizer = zarr.ProcessSynchronizer('../../argozarr/argodask1.sync')
compressor = zarr.Blosc(cname='zstd', clevel=3, shuffle=2)
zarr_path =  '../../argozarr/argo_dask.zarr'
data =  preproc(xr.open_dataset(files[200]))
data =data.chunk({'N_PROF':10000,'N_LEVELS':2000})
encoding = {vname: {'compressor': compressor} for vname in data.variables}
data.to_zarr(zarr_path,mode='w',encoding=encoding)




In [54]:

@dask.delayed
def write_key(key,data,store):
    store[key].append(data[key])


d =[]

for file in files[0:400]:
    d.append(process_float(file))
results = dask.compute(*d)
print('file opend')
t =xr.concat(results,dim='N_PROF')    
t=t.chunk({'N_PROF':1000,'N_LEVELS':3000})

   

file opend


In [55]:
os.environ['BLOSC_NOLOCK'] = "1"

In [56]:
import zarr
zarr_path =  '../../argozarr/argo_dask.zarr'
import numcodecs
numcodecs.blosc.use_threads = False
synchronizer = zarr.ProcessSynchronizer('../../argozarr/argodask1.sync')
compressor = zarr.Blosc(cname='zstd', clevel=3, shuffle=2)
zarr_path =  '../../argozarr/argo_dask.zarr'
encoding = {vname: {'compressor': compressor} for vname in t.variables}
d =t.to_zarr(zarr_path,mode='a',synchronizer =synchronizer,compute=False,append_dim='N_PROF')

In [57]:
%time d.compute()

CPU times: user 3min 19s, sys: 1min 35s, total: 4min 54s
Wall time: 2min 28s


In [45]:


ds1 = xr.open_zarr(zarr_path)
ds1.info()

xarray.Dataset {
dimensions:
	N_LEVELS = 2000 ;
	N_PROF = 64470 ;

variables:
	float64 CONFIG_MISSION_NUMBER(N_PROF) ;
	float64 CYCLE_NUMBER(N_PROF) ;
	<U2 DATA_CENTRE(N_PROF) ;
	<U1 DATA_MODE(N_PROF) ;
	<U4 DATA_STATE_INDICATOR(N_PROF) ;
	<U32 DC_REFERENCE(N_PROF) ;
	<U1 DIRECTION(N_PROF) ;
	<U32 FIRMWARE_VERSION(N_PROF) ;
	<U32 FLOAT_SERIAL_NO(N_PROF) ;
	float64 JULD(N_PROF) ;
	float64 JULD_LOCATION(N_PROF) ;
	<U1 JULD_QC(N_PROF) ;
	float64 LATITUDE(N_PROF) ;
	float64 LONGITUDE(N_PROF) ;
	<U64 PI_NAME(N_PROF) ;
	<U8 PLATFORM_NUMBER(N_PROF) ;
	<U32 PLATFORM_TYPE(N_PROF) ;
	<U8 POSITIONING_SYSTEM(N_PROF) ;
	<U1 POSITION_QC(N_PROF) ;
	float64 PRES(N_PROF, N_LEVELS) ;
	float64 PRES_ADJUSTED(N_PROF, N_LEVELS) ;
	float64 PRES_ADJUSTED_ERROR(N_PROF, N_LEVELS) ;
	<U1 PRES_ADJUSTED_QC(N_PROF, N_LEVELS) ;
	<U1 PRES_QC(N_PROF, N_LEVELS) ;
	<U1 PROFILE_PRES_QC(N_PROF) ;
	<U1 PROFILE_PSAL_QC(N_PROF) ;
	<U1 PROFILE_TEMP_QC(N_PROF) ;
	<U64 PROJECT_NAME(N_PROF) ;
	float64 PSAL(N_PROF, N_LEVELS) ;
	f